Merging the scraping script and the categories script.

And clean up with individual functions that are reused

In [1]:
path = '/home/bfh/irsed/daten/FIFA/'

In [2]:
from bs4 import BeautifulSoup
import requests

In [13]:
years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974,
         1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014,
         2018, 2022]
# years = [1958]

In [4]:
def create_field(document, fieldname, text):
    if len(text.strip()) > 0:
        field = xml.new_tag('field', attrs = {'name':fieldname})
        field.string = text
        document.append(field)

In [5]:
def find_h1(soup, document):
    h1 = soup.find('span', class_='mw-page-title-main').string
    create_field(document, "title", h1)

In [6]:
def clean_category(content):
    # Footnotes are in square brackets and will be removed
    try:
        start = content.index('[')  #exption, if missing
        end = content.index(']')
        text = content[0:start]
        counter = content[start+1:end]
        try:
            z = int(counter)
            content = text
            # skip footnote e.g. [42]
        except:
            pass
    except:
        pass
    # Information after the opening round brackets is no longer relevant
    try:
        start = content.index('(')
        content = content[0:start]
    except:
        pass
    return content.strip()

In [7]:
def one_document(year, document, categories, fields):
    # get the docuent again
    web = f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'
    response = requests.get(web)
    content = response.text
    soup = BeautifulSoup(content, 'lxml')
    do_text_fields(soup, document)
    do_category(soup, document, categories)
    do_fields(soup, fields)

In [8]:
def do_fields(soup, fields):
    bodies = soup.find_all('div', class_='mw-parser-output')
    if (len(bodies) != 1):
        print(f"Body text missing or not unique: year={year} count={len(bodies)}")
    for body in bodies:
        for child in body.children:
            if child.name == 'h2':
                h2 = child.find('span', class_='mw-headline').string
                # fields(h2)
                fields.add(h2.lower().replace(" ","_"))

In [9]:
def do_text_fields(soup, document):
    text = ""
    # Find h1
    find_h1(soup, document)
    # find all other fields
    bodies = soup.find_all('div', class_='mw-parser-output')
    if (len(bodies) != 1):
        print(f"Body does not exist or is not unique={year} count={len(bodies)}")
    fieldname = 'description' #first field
    for body in bodies:
        for child in body.children:
            if child.name == 'div':
                if child.has_attr("class") and "mw-heading2" in child["class"]:
                    # print("**********************************")
                    create_field(document, fieldname, text)
                    text = ""
                    # print (f'h2: {child.name}')
                    h2 = child.find("h2").getText()    
                    print(h2)
                    fieldname = h2.lower().replace(" ","_")
                    # print(fieldname)
            elif child.name == 'p':
                #print (f'p: {child.name}')
                # text = ""
                is_footnote = False;
                for elt in child.strings:
                    if elt.startswith('['):
                        is_footnote = True
                    if not is_footnote:
                            text += elt
                    if elt.startswith(']'):
                        is_footnote = False
                
def find_h1(soup, document):
    h1 = soup.find('span', class_='mw-page-title-main').string
    create_field(document, "title", h1)


In [10]:
def do_category(soup, document, categories):
    lines = soup.find('table', class_='infobox')
    trs = lines.find_all('tr')
    header = ""
    for tr in trs:
        label = tr.find('th', class_='infobox-label')
        content = tr.find('td', class_='infobox-data')
        newheader = tr.find('th', class_='infobox-header')
        if newheader != None:
            header = newheader.getText()
        if label != None:
            label = label.getText()
            content = tr.find('td', class_='infobox-data')
            text = ""
            if content != None:
                content = content.getText()
            content = clean_category(content)
            category = header + '_' + label
            category= category.lower().replace(" ","_").replace("(","").replace(")","").replace(u'\xa0', u'_')
            categories.add(category)
            create_field(document, category, content)

In [11]:
# write the output to a file
from bs4.formatter import XMLFormatter
def write_xml(pfad):
    with open(pfad, "w") as f:
        output = str(xml)
        output.replace('\n',' ')
        print(output)
        f.write(output)

In [14]:
categories = set()
fields = set()
fields.add('year')
fields.add('description')

xml = BeautifulSoup(features='xml')
add = xml.new_tag('add')
xml.append(add)

# create a new doc-tab for each year
for year in years:
    print(year)
    document = xml.new_tag('doc')
    create_field(document, "year", str(year))   # generate id field
    one_document(year, document, categories, fields)
    xml.add.append( document)


write_xml(path+"fifa_with_categories.xml")


1930
Participants
Venues
Match officials
Format and draw
Tournament summary
Group stage
Knockout stage
Goalscorers
FIFA retrospective ranking
See also
Notes
References
External links
1934
Host selection
Qualification and participants
Venues
Format
Summary
Squads
Final tournament
Goalscorers
FIFA retrospective ranking
References
Bibliography
External links
1938
Host selection
Qualification
Format
Summary
Venues
Squads
Final tournament
Goalscorers
FIFA retrospective ranking
Footnotes
References
External links
1950
Host selection
Qualification
Format
Summary
Venues
Squads
Match officials
Group stage
Final round
Goalscorers
FIFA retrospective ranking
Footnotes
Bibliography
External links
1954
Host selection
Qualification
Summary
Venues
Squads
Match officials
Group stage
Knockout stage
Goalscorers
FIFA retrospective ranking
In film
References
External links
1958
Host selection
Qualification
Format
Summary
Venues
Match officials
Seeding
Squads
Group stage
Knockout stage
Goalscorers
All-Star 